In [86]:
import pandas as pd

df = pd.read_csv('ls_new.csv')

df

/var/folders/fx/cj87kbwd5m3351r36_y81c440000gn/T/ipykernel_7633/4175615603.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ls_new.csv')


,Date,ReceiptNumber,LineType,CustomerCode,CustomerName,Note,Quantity,Subtotal,SalesTax,Discount,Loyalty,Total,Paid,Details,Register,User,Status,Sku,AccountCodeSale,AccountCodePurchase
0,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,3.46957,0.52044,0.0,0.0,3.99001,NaN,Edible Funny Money 14g,NaN,NaN,NaN,8713600500826,NaN,NaN
1,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,3.46957,0.52044,0.0,0.0,3.99001,NaN,Edible Funny Money 14g,NaN,NaN,NaN,8713600500826,NaN,NaN
2,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,1.73913,0.26087,0.0,0.0,2.00000,NaN,Sante Bar 25g (Whittakers) / Dark Single,NaN,NaN,NaN,94314281,NaN,NaN
3,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,4.33913,0.65087,0.0,0.0,4.99000,NaN,Fruit Flavoured Bubblegum 100g / Assorted,NaN,NaN,NaN,9417269128856,NaN,NaN
4,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,7.81739,1.17261,0.0,0.0,8.99000,NaN,Freeze Dried Original Fruit Skittles 100g,NaN,NaN,NaN,9417269125497,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113850,2024-05-31 09:33:08,112554,Sale Line,NaN,NaN,NaN,5.0,2.60870,0.39130,1.0,0.0,3.00000,NaN,Dobson Mega Lollipop Singles 22g / Tutti-frutti,NaN,NaN,NaN,11842,NaN,NaN
113851,2024-05-31 09:31:15,112553,Sale Line,NaN,NaN,NaN,1.0,0.43478,0.06522,0.0,0.0,0.50000,NaN,Wacky Tape / Strawberry 16g,NaN,NaN,NaN,8964000979778,NaN,NaN
113852,2024-05-31 09:31:15,112553,Sale Line,NaN,NaN,NaN,1.0,0.43478,0.06522,0.0,0.0,0.50000,NaN,Wacky Tape / Blueberry 15g,NaN,NaN,NaN,8964000979761,NaN,NaN
113853,2024-05-31 09:31:15,112553,Sale Line,NaN,NaN,NaN,1.0,0.26087,0.03913,0.0,0.0,0.30000,NaN,Rockets (Single),NaN,NaN,NaN,958641,NaN,NaN


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113855 entries, 0 to 113854
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Date                 113855 non-null  object 
 1   ReceiptNumber        113855 non-null  int64  
 2   LineType             113855 non-null  object 
 3   CustomerCode         0 non-null       float64
 4   CustomerName         0 non-null       float64
 5   Note                 1 non-null       object 
 6   Quantity             113855 non-null  float64
 7   Subtotal             113855 non-null  float64
 8   SalesTax             113855 non-null  float64
 9   Discount             113855 non-null  float64
 10  Loyalty              113855 non-null  float64
 11  Total                113855 non-null  float64
 12  Paid                 0 non-null       float64
 13  Details              113855 non-null  object 
 14  Register             0 non-null       float64
 15  User             

In [88]:
# Convert 'date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [89]:
#checking if it changed to datetime
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113855 entries, 0 to 113854
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date                 113855 non-null  datetime64[ns]
 1   ReceiptNumber        113855 non-null  int64         
 2   LineType             113855 non-null  object        
 3   CustomerCode         0 non-null       float64       
 4   CustomerName         0 non-null       float64       
 5   Note                 1 non-null       object        
 6   Quantity             113855 non-null  float64       
 7   Subtotal             113855 non-null  float64       
 8   SalesTax             113855 non-null  float64       
 9   Discount             113855 non-null  float64       
 10  Loyalty              113855 non-null  float64       
 11  Total                113855 non-null  float64       
 12  Paid                 0 non-null       float64       
 13  Details       

In [90]:
df.head()

,Date,ReceiptNumber,LineType,CustomerCode,CustomerName,Note,Quantity,Subtotal,SalesTax,Discount,Loyalty,Total,Paid,Details,Register,User,Status,Sku,AccountCodeSale,AccountCodePurchase
0,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,3.46957,0.52044,0.0,0.0,3.99001,NaN,Edible Funny Money 14g,NaN,NaN,NaN,8713600500826,NaN,NaN
1,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,3.46957,0.52044,0.0,0.0,3.99001,NaN,Edible Funny Money 14g,NaN,NaN,NaN,8713600500826,NaN,NaN
2,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,1.73913,0.26087,0.0,0.0,2.00000,NaN,Sante Bar 25g (Whittakers) / Dark Single,NaN,NaN,NaN,94314281,NaN,NaN
3,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,4.33913,0.65087,0.0,0.0,4.99000,NaN,Fruit Flavoured Bubblegum 100g / Assorted,NaN,NaN,NaN,9417269128856,NaN,NaN
4,2024-01-21 16:59:27,87445,Sale Line,NaN,NaN,NaN,1.0,7.81739,1.17261,0.0,0.0,8.99000,NaN,Freeze Dried Original Fruit Skittles 100g,NaN,NaN,NaN,9417269125497,NaN,NaN


In [91]:
# Sort the DataFrame by 'date' in ascending order
df = df.sort_values(by='Date')

In [92]:
df

,Date,ReceiptNumber,LineType,CustomerCode,CustomerName,Note,Quantity,Subtotal,SalesTax,Discount,Loyalty,Total,Paid,Details,Register,User,Status,Sku,AccountCodeSale,AccountCodePurchase
2050,2024-01-01 10:02:00,83816,Sale Line,NaN,NaN,NaN,3.0,17.39130,2.60868,3.97002,0.0,19.99998,NaN,Airplane Seconds (Manufacturers Clearance) / 500g,NaN,NaN,NaN,9.41727E+12,NaN,NaN
2049,2024-01-01 10:02:00,83816,Sale Line,NaN,NaN,NaN,1.0,0.26087,0.03913,0.00000,0.0,0.30000,NaN,Paper Checkout Bags,NaN,NaN,NaN,9.31071E+12,NaN,NaN
2048,2024-01-01 10:11:00,83817,Sale Line,NaN,NaN,NaN,1.0,6.52174,0.97826,0.00000,0.0,7.50000,NaN,Simpkin Tins / Orange/lemon/grapefruit 200g,NaN,NaN,NaN,5.01156E+12,NaN,NaN
2047,2024-01-01 10:11:00,83817,Sale Line,NaN,NaN,NaN,1.0,5.79710,0.86956,1.32334,0.0,6.66666,NaN,Orange & Lemon Slices 500g (Manufacturers Clea...,NaN,NaN,NaN,9.41727E+12,NaN,NaN
2046,2024-01-01 10:11:00,83817,Sale Line,NaN,NaN,NaN,1.0,5.79710,0.86956,1.32334,0.0,6.66666,NaN,Snakes (Manufacturers Clearance) / 500g,NaN,NaN,NaN,9.41727E+12,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112686,2024-05-31 17:39:24,112784,Sale Line,NaN,NaN,NaN,1.0,8.68696,1.30304,0.00000,0.0,9.99000,NaN,Russian Fudge Box Mary Gray 150g,NaN,NaN,NaN,9418353162046,NaN,NaN
112684,2024-05-31 17:45:56,112785,Sale Line,NaN,NaN,NaN,1.0,4.78261,0.71739,0.00000,0.0,5.50000,NaN,Aniseed Wheels / 6's/110g,NaN,NaN,NaN,9417169102680,NaN,NaN
112683,2024-05-31 17:45:56,112785,Sale Line,NaN,NaN,NaN,1.0,1.21739,0.18261,0.00000,0.0,1.40000,NaN,Licorice Log Sherbet Fizz / Raspberry 40g,NaN,NaN,NaN,9418315127298,NaN,NaN
112685,2024-05-31 17:45:56,112785,Sale Line,NaN,NaN,NaN,1.0,6.94783,1.04217,0.00000,0.0,7.99000,NaN,Aniseed Balls (Old Fashioned) / 300g,NaN,NaN,NaN,9417169009057,NaN,NaN


### Creating a subset for school holidays

In [93]:
# Create subset for dates between 13th April and 28th April
start_date = '2024-04-13'
end_date = '2024-04-28'

school_holidays = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]


In [94]:
school_holidays

,Date,ReceiptNumber,LineType,CustomerCode,CustomerName,Note,Quantity,Subtotal,SalesTax,Discount,Loyalty,Total,Paid,Details,Register,User,Status,Sku,AccountCodeSale,AccountCodePurchase
73178,2024-04-13 09:03:14,102147,Sale Line,NaN,NaN,NaN,1.0,2.43478,0.36522,0.00000,0.0,2.80000,NaN,Dr Bugs Popcorn 150g / Double Caramel,NaN,NaN,NaN,9414066900906,NaN,NaN
73177,2024-04-13 09:18:10,102148,Sale Line,NaN,NaN,NaN,1.0,5.79710,0.86956,1.32334,0.0,6.66666,NaN,Terrific Party Mix 1kg (Manufacturers Clearance),NaN,NaN,NaN,6923431513250,NaN,NaN
73176,2024-04-13 09:18:10,102148,Sale Line,NaN,NaN,NaN,1.0,5.79710,0.86956,1.32334,0.0,6.66666,NaN,Terrific Party Mix 1kg (Manufacturers Clearance),NaN,NaN,NaN,6923431513250,NaN,NaN
73175,2024-04-13 09:18:10,102148,Sale Line,NaN,NaN,NaN,1.0,5.79710,0.86956,1.32334,0.0,6.66666,NaN,Terrific Sour Party Mix 1kg (Manufactures Clea...,NaN,NaN,NaN,6923431513243,NaN,NaN
73174,2024-04-13 09:24:11,102149,Sale Line,NaN,NaN,NaN,1.0,0.43478,0.06522,0.00000,0.0,0.50000,NaN,Diamond Ring Pop 12g,NaN,NaN,NaN,9315312113401,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86485,2024-04-27 17:57:56,105911,Sale Line,NaN,NaN,NaN,1.0,0.52174,0.07826,0.20000,0.0,0.60000,NaN,Trolli Gummy Takeaways / Pizza 15.5g (Single),NaN,NaN,NaN,6925425473182,NaN,NaN
86484,2024-04-27 17:57:56,105911,Sale Line,NaN,NaN,NaN,1.0,0.52174,0.07826,0.20000,0.0,0.60000,NaN,Zombie Chews 28g / Sour Blue Raspberry,NaN,NaN,NaN,9338329000465,NaN,NaN
86483,2024-04-27 17:57:56,105911,Sale Line,NaN,NaN,NaN,1.0,0.52174,0.07826,0.20000,0.0,0.60000,NaN,Zombie Chews 28g / Sour Blue Raspberry,NaN,NaN,NaN,9338329000465,NaN,NaN
86482,2024-04-27 17:57:56,105911,Sale Line,NaN,NaN,NaN,1.0,0.52174,0.07826,0.20000,0.0,0.60000,NaN,Zombie Chews 28g / Sour Blue Raspberry,NaN,NaN,NaN,9338329000465,NaN,NaN


In [95]:
#trying to find the number of transactions during last school holidays
school_holidays['ReceiptNumber'].nunique()

3576

In [96]:
items_to_remove = ['Paper Checkout Bags','Discount', 'Clearance']

filtered_df = school_holidays[~school_holidays['Details'].isin(items_to_remove)]

In [97]:
filtered_df.shape

(15514, 20)

In [100]:
# Ensure 'Quantity' is integer and handle NaNs
filtered_df['Quantity'] = filtered_df['Quantity'].fillna(0).astype(int)

# Repeat 'Details' by 'Quantity' to calculate the real item quantity sold in each row
repeated_details = filtered_df.apply(lambda row: [row['Details']] * row['Quantity'], axis=1)

# Flatten the list of lists
flattened_details = [item for sublist in repeated_details for item in sublist]

# Calculate value counts
details_counts = pd.Series(flattened_details).value_counts().reset_index()

details_counts.columns = ['NameofProduct', 'PurchaseFrequencyDuringHolidays']

/var/folders/fx/cj87kbwd5m3351r36_y81c440000gn/T/ipykernel_7633/2534716358.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Quantity'] = filtered_df['Quantity'].fillna(0).astype(int)


In [101]:
details_counts['PurchaseFrequencyDuringHolidays'].sum()

16412

In [ ]:
details_counts.to_csv('HighestProductsSoldDuringHolidays.csv')

In [ ]:
sales_line_df = filtered_df[['ReceiptNumber', 'Details']]

In [ ]:
# Convert the transaction data into a one-hot encoded format
# Get unique items
unique_items = sales_line_df['Details'].unique()
# Transform to Table Format (One-Hot Encoding)
df_retail_txn_table = sales_line_df.pivot_table(index='ReceiptNumber', columns='Details', aggfunc='size', fill_value=0)
df_retail_txn_table

Details,$2 Pick 'n' Mix Bag,A & W Root Beer 355ml,ABC Letters Dextrose 100g,Acid Drops (Old Fashioned) / 300g,Aero Bar 36g / Peppermint,Aero Bar 90g / Honeycomb,After Shocks Popping Candy / Blue Raspberry,After Shocks Popping Candy / Cherry,After Shocks Popping Candy / Cotton Candy,After Shocks Popping Candy / Grape,...,York Peppermint Patties 13g,Yummy Tarts 100g,Zappo Millions Rainbow 75g,Zappo Stick Packs 29g / Blue Raspberry,Zappo Stick Packs 29g / Sour Strawberry,Zappo Stick Packs 29g / Watermelon,Zombie Chews 28g / Cola,Zombie Chews 28g / Grape,Zombie Chews 28g / Sour Blue Raspberry,Zoolu Pet Chew Balls 20g
ReceiptNumber,,,,,,,,,,,,,,,,,,,,,
102147,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105908,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# You can see we have the count, which may be >1 for some cases. We'll change this to binary to align with the format.
df_retail_txn_table = (df_retail_txn_table > 0).astype(int) # This one sets all 1+ values to True & convert it to 1
df_retail_txn_table

Details,$2 Pick 'n' Mix Bag,A & W Root Beer 355ml,ABC Letters Dextrose 100g,Acid Drops (Old Fashioned) / 300g,Aero Bar 36g / Peppermint,Aero Bar 90g / Honeycomb,After Shocks Popping Candy / Blue Raspberry,After Shocks Popping Candy / Cherry,After Shocks Popping Candy / Cotton Candy,After Shocks Popping Candy / Grape,...,York Peppermint Patties 13g,Yummy Tarts 100g,Zappo Millions Rainbow 75g,Zappo Stick Packs 29g / Blue Raspberry,Zappo Stick Packs 29g / Sour Strawberry,Zappo Stick Packs 29g / Watermelon,Zombie Chews 28g / Cola,Zombie Chews 28g / Grape,Zombie Chews 28g / Sour Blue Raspberry,Zoolu Pet Chew Balls 20g
ReceiptNumber,,,,,,,,,,,,,,,,,,,,,
102147,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105908,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# We will first convert the DataFrame to have Boolean (True/False) instead of 1/0
df_retail_final=(df_retail_txn_table > 0)
df_retail_final

Details,$2 Pick 'n' Mix Bag,A & W Root Beer 355ml,ABC Letters Dextrose 100g,Acid Drops (Old Fashioned) / 300g,Aero Bar 36g / Peppermint,Aero Bar 90g / Honeycomb,After Shocks Popping Candy / Blue Raspberry,After Shocks Popping Candy / Cherry,After Shocks Popping Candy / Cotton Candy,After Shocks Popping Candy / Grape,...,York Peppermint Patties 13g,Yummy Tarts 100g,Zappo Millions Rainbow 75g,Zappo Stick Packs 29g / Blue Raspberry,Zappo Stick Packs 29g / Sour Strawberry,Zappo Stick Packs 29g / Watermelon,Zombie Chews 28g / Cola,Zombie Chews 28g / Grape,Zombie Chews 28g / Sour Blue Raspberry,Zoolu Pet Chew Balls 20g
ReceiptNumber,,,,,,,,,,,,,,,,,,,,,
102147,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
102148,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
102149,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
102150,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
102151,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105907,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
105908,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
105909,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import seaborn as sns

In [ ]:
# Apply the Apriori algorithm to find frequent itemsets
min_support = 0.003  # Minimum support threshold - change this number and run again
frequent_itemsets = apriori(df_retail_final, min_support=min_support, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.010753,(Airhead Bars 15g)
1,0.003396,(Alien Ring Pop 5g)
2,0.003962,(Aniseed Wheels / 6's/110g)
3,0.004527,(Au'some Fruit Blasts 52g)
4,0.005376,(Barnetts Mega Sours 100g / Sour Apple)
...,...,...
599,0.003679,"(Wacky Tape / Blueberry 15g, Busters, Trolli G..."
600,0.006508,"(Busters, Wacky Tape / Strawberry 16g, Trolli ..."
601,0.003113,"(Wacky Tape / Blueberry 15g, Wacky Tape / Stra..."
602,0.004527,"(JoJo Jelly Sour Sticks / Apple, JoJo Jelly So..."


In [ ]:
# Discover association rules
min_confidence = 0.01  # Minimum confidence threshold
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_confidence)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Airhead Bars 15g),(Trolli Gummy Takeaways / Pizza 15.5g (Single)),0.010753,0.045557,0.003113,0.289474,6.354037,0.002623,1.343290,0.851779
1,(Trolli Gummy Takeaways / Pizza 15.5g (Single)),(Airhead Bars 15g),0.045557,0.010753,0.003113,0.068323,6.354037,0.002623,1.061792,0.882840
2,(Airhead Bars 15g),(Wacky Sour Filled Chews 24g),0.010753,0.022637,0.003396,0.315789,13.950000,0.003152,1.428453,0.938406
3,(Wacky Sour Filled Chews 24g),(Airhead Bars 15g),0.022637,0.010753,0.003396,0.150000,13.950000,0.003152,1.163820,0.949817
4,(Buzzy Crock Candy 4g),(Big Blue Candy),0.016978,0.013865,0.004810,0.283333,20.434694,0.004575,1.376002,0.967490
...,...,...,...,...,...,...,...,...,...,...
899,"(Trolli Gummy Takeaways / Burger 9g (Single), ...","(Wacky Tape / Blueberry 15g, Wacky Tape / Stra...",0.036786,0.011885,0.003396,0.092308,7.767033,0.002958,1.088602,0.904524
900,(Wacky Tape / Blueberry 15g),"(Wacky Tape / Strawberry 16g, Trolli Gummy Tak...",0.024052,0.011602,0.003396,0.141176,12.168723,0.003117,1.150875,0.940442
901,(Wacky Tape / Strawberry 16g),"(Wacky Tape / Blueberry 15g, Trolli Gummy Take...",0.036786,0.005659,0.003396,0.092308,16.310769,0.003187,1.095460,0.974540
902,(Trolli Gummy Takeaways / Burger 9g (Single)),"(Wacky Tape / Blueberry 15g, Wacky Tape / Stra...",0.051783,0.004810,0.003396,0.065574,13.631630,0.003146,1.065027,0.977246


In [ ]:
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Airhead Bars 15g),(Trolli Gummy Takeaways / Pizza 15.5g (Single)),0.010753,0.045557,0.003113,0.289474,6.354037,0.002623,1.343290,0.851779
1,(Trolli Gummy Takeaways / Pizza 15.5g (Single)),(Airhead Bars 15g),0.045557,0.010753,0.003113,0.068323,6.354037,0.002623,1.061792,0.882840
2,(Airhead Bars 15g),(Wacky Sour Filled Chews 24g),0.010753,0.022637,0.003396,0.315789,13.950000,0.003152,1.428453,0.938406
3,(Wacky Sour Filled Chews 24g),(Airhead Bars 15g),0.022637,0.010753,0.003396,0.150000,13.950000,0.003152,1.163820,0.949817
4,(Buzzy Crock Candy 4g),(Big Blue Candy),0.016978,0.013865,0.004810,0.283333,20.434694,0.004575,1.376002,0.967490
5,(Big Blue Candy),(Buzzy Crock Candy 4g),0.013865,0.016978,0.004810,0.346939,20.434694,0.004575,1.505253,0.964436
6,(Big Blue Candy),(Ghouliez Liquid Centre / Blueberry),0.013865,0.017261,0.003396,0.244898,14.188023,0.003156,1.301465,0.942587
7,(Ghouliez Liquid Centre / Blueberry),(Big Blue Candy),0.017261,0.013865,0.003396,0.196721,14.188023,0.003156,1.227637,0.945844
8,(Big Blue Candy),(Razzoo 11g (Mayceys) / Grape),0.013865,0.017544,0.003113,0.224490,12.795918,0.002869,1.266851,0.934812
9,(Razzoo 11g (Mayceys) / Grape),(Big Blue Candy),0.017544,0.013865,0.003113,0.177419,12.795918,0.002869,1.198830,0.938312


In [ ]:
# Let us compile a final list with some filter, calculation & sorting
final_rules=rules[(rules['lift']>1)&(rules['support']>=0.001)&(rules['confidence']<=0.09)]
# Determine number of items in X => predicting number of items in Y
final_rules=final_rules.copy() # this step creates an independent list instead of a view on rules from above
# We capture the number of items in the list of each antecedent/consequent set using 'len' function for each row
final_rules['antecedent_count']=final_rules['antecedents'].apply(len)
final_rules['consequent_count']=final_rules['consequents'].apply(len) #X

# Apply some rounding and sorting on Lift
final_rules=round(final_rules,2).sort_values(by=['lift', 'confidence'], ascending=[False, False])

# Preserve relevant columns only
final_rules=final_rules[['antecedents', 'antecedent_count', 'consequents', 'consequent_count', 'support', 'confidence', 'lift']]
final_rules

,antecedents,antecedent_count,consequents,consequent_count,support,confidence,lift
623,(Trolli Gummy Takeaways / Hot Dog 9g (Single)),1,"(Buzzy Crock Candy 4g, Trolli Gummy Takeaways ...",2,0.0,0.07,20.19
645,(Trolli Gummy Takeaways / Hot Dog 9g (Single)),1,"(Trolli Gummy Takeaways / Burger 9g (Single), ...",2,0.0,0.07,18.64
622,(Trolli Gummy Takeaways / Burger 9g (Single)),1,"(Buzzy Crock Candy 4g, Trolli Gummy Takeaways ...",2,0.0,0.07,17.93
520,(Wacky Tape / Strawberry 16g),1,"(Gummy Ball 3D / Football, Busters)",2,0.0,0.08,16.61
831,(Trolli Gummy Takeaways / Burger 9g (Single)),1,"(Busters, Trolli Gummy Takeaways / Hot Dog 9g ...",3,0.0,0.06,16.34
...,...,...,...,...,...,...,...
467,(Trolli Gummy Takeaways / Pizza 15.5g (Single)),1,(Wacky Tape / Blueberry 15g),1,0.0,0.08,3.36
159,(Trolli Gummy Takeaways / Pizza 15.5g (Single)),1,(Ghouliez Chews 28g / Apple),1,0.0,0.07,3.29
438,(Trolli Gummy Takeaways / Burger 9g (Single)),1,(Wacky Sour Filled Chews 24g),1,0.0,0.07,3.14
191,(Trolli Gummy Takeaways / Hot Dog 9g (Single)),1,(Ghouliez Chews 28g / Strawberry),1,0.0,0.06,3.09


In [ ]:
final_rules.to_csv('school_holidays.csv')